In [7]:
#importing libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy.stats as stats
import sklearn as sk
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split  
from surprise import accuracy
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV

# <span style='color:#F76241'>|</span> Group 4 - Systems Development for Marketing </b>
### Building a Recommended System for Netflix and Jester

## <b>1<span style='color:#F76241'>|</span> Introduction

**<span style='color:#F76241'>Main Research question:</span>** </br>
How can we use the Netflix Prize dataset to build a recommender system that accurately predicts user ratings for movies and TV shows, and what are the most effective algorithms and techniques for doing so? </br>

</br>**<span style='color:#F76241'>Sub Research questions:</span>** </br>

What are the most effective algorithms and techniques for predicting user ratings based on the Netflix Prize dataset? </br>

How can we evaluate the accuracy and effectiveness of different recommender system algorithms using the Netflix Prize dataset? </br>

What are the most important features and variables to consider when building a recommender system using the Netflix Prize dataset? </br>


</div>

## <b>1.1<span style='color:#F76241'>|</span> About the Dataset

The Netflix Prize dataset is a large dataset of movie ratings that was released by Netflix in 2006. The dataset consists of about 100,000,000 ratings for 17,770 movies given by 480,189 users1. Each rating in the training dataset consists of four entries: user, movie, date of grade, grade. Users and movies are represented with integer IDs, while ratings range from 1 to 51. The goal of the Netflix Prize was to develop a recommendation algorithm that could predict user ratings for movies and TV shows with greater accuracy than Netflix’s existing algorithm, Cinematch2.

In [22]:
#Merging the training dataset, code adopted from https://www.kaggle.com/code/elemento/netflix-prizedata
if not os.path.isfile('C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/data.csv'):
    # Create a file 'data.csv' before reading it
    # Read all the files in the dataset and store them in one big file ('data.csv')
    # We're reading from each of the four files and appending each rating to a global file 'data.csv'
    
    data= open('data.csv', mode='w')
    
    row = []
    files = [
        'C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_1.txt',
        'C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_2.txt', 
        'C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_3.txt', 
        'C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_4.txt']
    for file in files:
        print("Reading ratings from {}\n".format(file))
        with open(file) as f:
            for line in f: 
                line = line.strip()
                if line.endswith(':'):
                    # All below are ratings for this movie, until another movie appears.
                    movie_id = line.replace(':', '')
                else:
                    row = [x for x in line.split(',')]
                    row.insert(0, movie_id)
                    data.write(','.join(row))
                    data.write('\n')
    data.close()

Reading ratings from C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_1.txt

Reading ratings from C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_2.txt

Reading ratings from C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_3.txt

Reading ratings from C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/combined_data_4.txt



In [23]:
# print("Creating the dataframe from data.csv file")
df = pd.read_csv('C:/Users/User/Dropbox/PC/Desktop/Masters DDB/Sysem development for marketing/Recsys Assignment/data.csv', sep=',', 
    names=['movie', 'user', 'rating', 'date'])
df.date = pd.to_datetime(df.date)

# We are arranging the ratings according to time-stamp(s)
print('Sorting the dataframe by Date')
df.sort_values(by='date', inplace=True)
print('Done')

Sorting the dataframe by Date
Done


In [24]:
df.head()

,movie,user,rating,date
56431994,10341,510180,4,1999-11-11
9056171,1798,510180,5,1999-11-11
58698779,10774,510180,3,1999-11-11
48101611,8651,510180,2,1999-11-11
81893208,14660,510180,2,1999-11-11
